<a href="https://colab.research.google.com/github/amorfinv/icrat_notebooks/blob/main/street_data_scenario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using street data in BlueSky


---


The goal of this notebook is to show how to use street data from OpenStreetMap to create routes for VTOL vehciles in constrained airspace. Constrained airspace is defined as airspace where aircraft must fly above the roads. Some potential reasons for flying in constrained airspace:


*   Avoidance of static obstacles (buildings) in very low level (VLL) airspace.
*   Limit third party risk.
*   Privacy


>*It is more likely that a delivery drones or other vehicle will be able to fly above buildings in VLL airspace. For these cases it is useful it is possible to define geofenced areas (e.g. a park) using OpenStreetMap data. However, this is a topic for another notebook.*

Start by importing ```osmnx```, ```geopandas```, ```shapely```. Note that ```leafmap``` is only neeeded for visualizing maps in this notebook.

In [1]:
import subprocess

try:
    import osmnx as ox
except ImportError:
    print('Installing osmnx ...')
    # pip installing osmnx brings in geopandas, shapely, and numpy
    subprocess.check_call(["python", '-m', 'pip', 'install', 'osmnx'])
    import osmnx as ox

# Leafmap is only necessary for visualizing maps in the notebook
try:
    import leafmap
except ImportError:
    print('Installing leafmap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'leafmap'])
    import leafmap

import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon, MultiLineString
from shapely import ops

Installing osmnx ...
Installing leafmap ...


## Choose an area of interest: Downtown Tampa

A drone mission in downtown tampa will be prepared so it can be simulated by BlueSky. The drone should:

*   Perform vertical take-off and climb to 60 feet
*   Accelerate to 30 knots and begin following a route that remains above the street network.
*   Decelerate to 10 knots prior to turning to avoid overshooting.
*   Decelerate to 0 knots above destination.
*   Descend to 0 feet to finish mission.

For simplicity, the mission will assume that the origin and destination points are in street intersections.

In [2]:
# Show map of Tampa
map_args = {"center":(27.95056, -82.45656), "zoom":16, "height":"500px", "width":"500px"}
cartomap = {
    "url":"https://cartodb-basemaps-b.global.ssl.fastly.net/light_nolabels/{z}/{x}/{y}.png",
    "name":"CartoDB",
    "attribution":"CartoDB",
}
m = leafmap.Map(**map_args)
m.add_tile_layer(**cartomap)
display(m)

Map(center=[27.95056, -82.45656], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

## Select the origin and destination

The origin and destination are chose somwhere in Downtown Tampa. The mission will include turns because there is no straight line path that only goes above a street.




In [3]:
# Create a mission from pharmacy to courthouse
origin_point = Point(-82.45863, 27.94853)
destination_point = Point(-82.45413, 27.95102)

# dictionary for geodataframe
data = {'type': ['origin', 'destination'], 
        'geometry': [origin_point, destination_point],
        'latitude': [27.94853, 27.95102],
        'longitude': [-82.45863, -82.45413]
        }
gdf_od = gpd.GeoDataFrame(data, crs='epsg:4326')

# create map and add gdf
m = leafmap.Map(**map_args)
m.add_tile_layer(**cartomap)
m.add_points_from_xy(gdf_od, popup='type', layer_name='od', zoom_to_layer=False)
display(m)

Map(center=[27.95056, -82.45656], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

# Model the street network

It is helpful to model the street network as a Graph in which the intersections are **nodes** and the streets are **edges**. This will allow us to use typical path planning algorithms for greating the route from an origin to a destination node. ```osmnx``` uses OpenStreetMap data to create a ```networkx``` graph of the street network.

```osmnx``` accepts a polygon of an area of interest and returns a the graph object. This example will use a simple ```shapely``` ```Polygon()``` that encapsulates the nearby streets and intersections near the origin and destination.

In [4]:
# create a polygon of area of interest
p1 = Point(-82.46079, 27.95042)
p2 = Point(-82.45905, 27.94679)
p3 = Point(-82.45168, 27.94934)
p4 = Point(-82.45344, 27.95290)

polygon = Polygon([p1, p2, p3, p4, p1])

# Get a graph of the area
G = ox.graph_from_polygon(polygon, network_type='drive')

# Show map

# Convert to gdf for plotting
nodes, edges = ox.graph_to_gdfs(G)
nodes['osmid'] = nodes.index
nodes.reset_index(drop=True, inplace=True)

map_args = {"center":(27.95056, -82.45656), "zoom":17, "height":"800px", "width":"1000px"}
m = leafmap.Map(**map_args)
m.add_tile_layer(**cartomap)
m.add_points_from_xy(gdf_od, popup='type', layer_name='od', zoom_to_layer=False)
m.add_points_from_xy(nodes, x="x", y="y", layer_name='nodes', zoom_to_layer=False, popup='osmid')
m.add_gdf(edges, layer_name='edges', style={"color": "black", "weight": 3}, zoom_to_layer=False)

display(m)

Map(center=[27.95056, -82.45656], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

## Finding a route

The typical method of finding a route is to use the origin and destination nodes and returning a list of nodes that should be passed to reach the destination. ```osmnx``` has a function that finds the nearest nodes to a particular point.

The origin and destination nodes can be passed to any ```networkx``` path finding algorithm. This example uses ```osmnx.shortest_path``` which is an implemention of ```nx.shortest_path``` that uses Dijkstra's algorithm with the street length as weights.

In [5]:
# get a route from origin to destination
X = [origin_point.x, destination_point.x]
Y = [origin_point.y, destination_point.y]

# find the nearest nodes to the origin and destination points
orig_node, dest_node = ox.nearest_nodes(G, X, Y, return_dist=False)

# Get the route as a list of nodes
route = ox.shortest_path(G, orig_node, dest_node)

# Show map

# get gemetry of route and put in a gdf for drawing
route_geom = ox.utils_graph.get_route_edge_attributes(G, route, attribute="geometry")
multi_line = MultiLineString(route_geom)
route_gdf = gpd.GeoDataFrame(geometry=[ops.linemerge(multi_line)], crs="epsg:4326")

m = leafmap.Map(**map_args)
m.add_tile_layer(**cartomap)
m.add_points_from_xy(gdf_od, x="longitude", y="latitude", popup='type', layer_name='od', zoom_to_layer=False)
m.add_gdf(edges, layer_name='edges', style={"color": "black", "weight": 3}, zoom_to_layer=False)
m.add_gdf(route_gdf, layer_name='route', style={"color": "red", "weight": 6}, zoom_to_layer=False)
display(m)


Map(center=[27.95056, -82.45656], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

# Convert route to a BlueSky sceneario

There is an accompanying ```bstools``` module that comes with some functions that use the ```osmnx``` graph and route to create a scenario which can be used in BlueSky.

In [6]:
# now we can create a bluesky scenario with this route
import bstools

# create the scenario
lats, lons = bstools.get_lat_lon_from_osm_route(G, route)
turn_bool = bstools.get_turn_arrays(lats, lons)
scenario = bstools.create_scenario_text(lats, lons, turn_bool)

# save the scenario file
with open('D1.scn', "w") as f:
  f.write(scenario)

print(scenario)

# Allow BlueSky to understand 0 speed commands
00:00:00>CASMACHTHR 0.0

# Zoom and turn on tiledmap
00:00:00>ZOOM 350
00:00:00>VIS MAP TILEDMAP

# Create drone at height 0 speed 0 and pan to it
00:00:00>CRE D1 M600 27.9484372 -82.4582366 68.70043011899615 0 0
00:00:00>PAN D1

# Tell drone to climb to 60 feet
00:00:00>ALT D1 60

# At 60 feet drone should increase speed to 30 knots
00:00:00>D1 ATALT 60 SPD D1 30

# Add waypoints
00:00:00>ADDWAYPOINTS D1,27.9484652 -82.4581553,,30,FLYBY, 0,27.948628 -82.457683,,30,FLYBY, 0,27.9486836 -82.4575218,,30,FLYBY, 0,27.9487174 -82.4574238,,30,TURNSPD,10,27.9488104 -82.4574634,,30,FLYBY, 0,27.9490581 -82.4575684,,30,FLYBY, 0,27.949389 -82.4577099,,30,FLYBY, 0,27.949467 -82.457742,,30,TURNSPD,10,27.9495045 -82.457635,,30,FLYBY, 0,27.9497307 -82.4569898,,30,FLYBY, 0,27.949759 -82.456904,,30,FLYBY, 0,27.9497928 -82.4568055,,30,FLYBY, 0,27.949959 -82.45632,,30,FLYBY, 0,27.9500109 -82.4561667,,30,FLYBY, 0,27.950046 -82.456066,,30,FLYBY, 0,27.9500819 -8

## Summary

Below is all the code needed to write a simple scenario file that will work in BlueSky. Note that all of the ```leafmap``` code is gone because it was only used for visualization.

In [9]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point, Polygon

# create a polygon of area of interest
p1 = Point(-82.46079, 27.95042)
p2 = Point(-82.45905, 27.94679)
p3 = Point(-82.45168, 27.94934)
p4 = Point(-82.45344, 27.95290)

polygon = Polygon([p1, p2, p3, p4, p1])

# Get a graph of the area
G = ox.graph_from_polygon(polygon, network_type='drive')

### perform simplification of graph here ###

# Create a mission from pharmacy to courthouse
origin_point = Point(-82.45863, 27.94853)
destination_point = Point(-82.45413, 27.95102)

# get a route from origin to destination
X = [origin_point.x, destination_point.x]
Y = [origin_point.y, destination_point.y]

# find the nearest nodes to the origin and destination points
orig_node, dest_node = ox.nearest_nodes(G, X, Y, return_dist=False)

# Get the route as a list of nodes
route = ox.shortest_path(G, orig_node, dest_node)

# create the scenario
lats, lons = bstools.get_lat_lon_from_osm_route(G, route)
turn_bool = bstools.get_turn_arrays(lats, lons)
scenario = bstools.create_scenario_text(lats, lons, turn_bool)

# save the scenario file
with open('D1.scn', "w") as f:
  f.write(scenario)

print(scenario)

# Allow BlueSky to understand 0 speed commands
00:00:00>CASMACHTHR 0.0

# Zoom and turn on tiledmap
00:00:00>ZOOM 350
00:00:00>VIS MAP TILEDMAP

# Create drone at height 0 speed 0 and pan to it
00:00:00>CRE D1 M600 27.9484372 -82.4582366 68.70043011899615 0 0
00:00:00>PAN D1

# Tell drone to climb to 60 feet
00:00:00>ALT D1 60

# At 60 feet drone should increase speed to 30 knots
00:00:00>D1 ATALT 60 SPD D1 30

# Add waypoints
00:00:00>ADDWAYPOINTS D1,27.9484652 -82.4581553,,30,FLYBY, 0,27.948628 -82.457683,,30,FLYBY, 0,27.9486836 -82.4575218,,30,FLYBY, 0,27.9487174 -82.4574238,,30,TURNSPD,10,27.9488104 -82.4574634,,30,FLYBY, 0,27.9490581 -82.4575684,,30,FLYBY, 0,27.949389 -82.4577099,,30,FLYBY, 0,27.949467 -82.457742,,30,TURNSPD,10,27.9495045 -82.457635,,30,FLYBY, 0,27.9497307 -82.4569898,,30,FLYBY, 0,27.949759 -82.456904,,30,FLYBY, 0,27.9497928 -82.4568055,,30,FLYBY, 0,27.949959 -82.45632,,30,FLYBY, 0,27.9500109 -82.4561667,,30,FLYBY, 0,27.950046 -82.456066,,30,FLYBY, 0,27.9500819 -8

## Final remarks
In reality, the street graph may need some simplification before it can be used. This can include:

*   Merging parallel edges that represent the same street.
*   Remvoing roundabouts.
*   Modifying edge directionality.
*   many others...

The next notebook is about including geofences for cases where it is not necessary to follow streets in VLL.